In [7]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import pandas as pd
import torch

data = load_wine()
X = data.data

print(data.feature_names)

df = pd.DataFrame(X, columns=data.feature_names)
df.head()

y = df['alcohol']
df = df.drop('alcohol', axis=1)
df.head()

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [9]:
X_train = torch.tensor(X_train.values)
X_test = torch.tensor(X_test.values)
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

In [10]:
X_train.shape

torch.Size([142, 12])

# Neuronale Netzarchitektur

* Wir haben 12 Attribute / Spalten / Features
* Wir haben 142 Traingsbeispiele

Angenommen, ein HiddenLayer mit 100 Neuronen

* w1: von 12 auf 100
* w2: von 100 auf 1

In [11]:
# scaling the data
X_train = X_train.float()
X_test = X_test.float()
y_train = y_train.float()
y_test = y_test.float()

In [13]:
# creating the model
lr = 0.01
losses = []
epochs = 100

X = X_train
y = y_train

# initializing the weights and bias
w1 = torch.normal(size=(12, 100), std=1, mean=0, requires_grad=True) # 12 features, 100 neurons
b1 = torch.tensor(1.0, requires_grad=True) # 1 bias

w2 = torch.normal(size=(100, 1), std=1, mean=0, requires_grad=True) # 100 neurons, 1 output
b2 = torch.tensor(1.0, requires_grad=True) # 1 bias

In [14]:


def train(single_x, single_y):
    # forward pass
    z2 = single_x @ w1 + b1
    a2 = torch.relu(z2)
    z3 = a2 @ w2 + b2
    y_hat = torch.relu(z3)

    # calculating the loss
    loss = (single_y-y_hat)**2

    # backward pass
    loss.backward()

    # updating the weights and bias
    with torch.no_grad():
        w1 -= lr * w1.grad
        b1 -= lr * b1.grad
        w2 -= lr * w2.grad
        b2 -= lr * b2.grad

        w1.grad.zero_()
        b1.grad.zero_()
        w2.grad.zero_()
        b2.grad.zero_()

    return loss

from tqdm import tqdm

for epoch in tqdm(range(epochs)):
    lossesSum = 0
    for single_x, single_y in zip(X, y):
        lossesSum += train(single_x, single_y)
    losses.append(lossesSum/len(X))
    if len(losses) > 2 and losses[-1] > losses[-2]:
        break
    if torch.sum(w1) == 0 or torch.sum(w2) == 0:
        break

import matplotlib.pyplot as plt

plt.plot(losses)

  0%|          | 0/100 [00:00<?, ?it/s]


UnboundLocalError: local variable 'w1' referenced before assignment

In [15]:
w1

tensor([[-0.8855, -0.6057,  0.5473,  ...,  0.2874,  0.3535,  1.1763],
        [-0.5562, -1.0874,  0.0636,  ...,  1.5780, -0.0496,  0.3055],
        [-0.5475,  0.7025, -2.1277,  ...,  1.5865,  0.7488, -0.2740],
        ...,
        [ 0.2688,  0.3953, -0.2260,  ..., -3.1643, -1.1194,  1.1269],
        [ 0.0242, -0.9976, -0.8171,  ...,  0.5738,  0.3219, -0.7670],
        [-0.5758,  1.3587,  1.9837,  ...,  0.9194, -1.3842,  0.4010]],
       requires_grad=True)

In [16]:
single_x = X_test[0]
z2 = single_x @ w1 + b1

In [17]:
z2

tensor([ -442.5692,  1153.7201,  1764.9165,   755.8215,  1354.5923, -1436.0560,
        -1692.8647,    40.4034, -1857.6515,  -134.6633, -1311.7625, -1984.0797,
          166.7047, -1793.3156,  1678.4268,    40.1197,   663.0154,   831.0491,
         1070.2036,   130.4381,   727.6436,   159.1927,  -653.8372,   837.2643,
         -687.4496,  -146.0394,  -554.1072,  -101.4203, -1237.5100,  -747.0682,
           55.4925,   731.7524, -1032.7566,   269.0438,  -622.1618,  -435.1540,
         -198.0020,  -686.1747,  -732.3129,   -96.6018,  -339.6108,   996.5349,
         -286.2181,  1758.3984,  1261.0315,  -186.1330,    72.3644,   334.1699,
          984.6340,   608.5522,  -429.4486,  -309.5009,  1023.3293, -1371.1216,
         -181.1306,  -832.6940,  -882.9839,   828.4915,   128.0011,  1229.1248,
        -1170.8733,  -859.3722,   218.0862,   672.7506,  -372.7230,  -405.3913,
         1462.0317,  -496.1927,   861.2671,  -748.1891,  -194.3233,  1791.5846,
         -692.9407,  1083.3510,   548.31